In [1]:
# Import the necessary libraries.
import os

# Clone the repository.
os.system("git clone https://github.com/markl-a/LLM_relatedDemo.git")

# Change to the directory containing the code.
os.chdir("/content/LLM_relatedDemo/GaLore_Demo/GaLore-master")

In [3]:
!pip install -e .

Obtaining file:///content/LLM_relatedDemo/experiment/GaLore-master
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00

用法


```
from galore_torch import GaLoreAdamW, GaLoreAdamW8bit, GaLoreAdafactor
# define param groups as galore_params and non_galore_params
param_groups = [{'params': non_galore_params},
                {'params': galore_params, 'rank': 128, 'update_proj_gap': 200, 'scale': 0.25, 'proj_type': 'std'}]
optimizer = GaLoreAdamW(param_groups, lr=0.01)
```



請先登入 wandb : 請參考 [wandb使用教學](https://zhuanlan.zhihu.com/p/493093033)



In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 使用具有 24GB 記憶體的單一 GPU 訓練 7B 模型 (這邊在colab 是使用A100 )

要使用單一 GPU（例如 NVIDIA RTX 4090）訓練 7B 模型，您所需要做的就是指定--optimizer=galore_adamw8bit_per_layer，這會啟用GaLoreAdamW8bit每層權重更新。透過啟動檢查點，您可以將在 NVIDIA RTX 4090 上測試的批次大小保持為 16。

In [ ]:
# LLaMA-7B, 8-bit GaLore-Adam, single GPU, activation checkpointing
# bsz=16, 22.8G,
!torchrun --standalone --nproc_per_node 1 torchrun_main.py \
    --model_config configs/llama_7b.json \
    --lr 0.005 \
    --galore_scale 0.25 \
    --rank 1024 \
    --update_proj_gap 500 \
    --batch_size 16 \
    --total_batch_size 512 \
    --activation_checkpointing \
    --num_training_steps 150000 \
    --warmup_steps 15000 \
    --weight_decay 0 \
    --grad_clipping 1.0 \
    --dtype bfloat16 \
    --eval_every 1000 \
    --single_gpu \
    --optimizer galore_adamw8bit_per_layer

[2024-03-11 14:42:22,628] torch.distributed.run: [WARNING] master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
Starting script
2024-03-11 14:42:29.183 | INFO     | __main__:main:140 - Global rank 0, local rank 0, device: 0
2024-03-11 14:42:29.192 | INFO     | __main__:main:144 - Process group initialized
wandb: Currently logged in as: m4932981. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /content/LLM_relatedDemo/experiment/GaLore-master/wandb/run-20240311_144230-wibzfpu6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run serene-firebrand-2
wandb: ⭐️ View project at https://wandb.ai/m4932981/galore-c4
wandb: 🚀 View run at https://wandb.ai/m4932981/galore-c4/runs/wibzfpu6
2024-03-11 14:42:31.439 | INFO     | __main__:main:163 - Using dist with rank 0 (only rank 0 will log)
2024-03-11 14:42:31.440 | INFO     | __main__:main:164 - ************************

# **Benchmark 1：在C4資料集上預訓練LLaMA**

torchrun_main.py是使用 GaLore 在 C4 上訓練 LLaMA 模型的主要腳本。我們針對各種尺寸模型的基準腳本位於scripts/benchmark_c4資料夾中。例如，要在 C4 上訓練 60m 模型，請執行以下操作：

In [ ]:
# LLaMA-60M, GaLore-Adam, 1 A100, 1 Node
!torchrun --standalone --nproc_per_node 1 torchrun_main.py \
    --model_config configs/llama_60m.json \
    --lr 0.01 \
    --galore_scale 0.25 \
    --rank 128 \
    --update_proj_gap 200 \
    --batch_size 256 \
    --total_batch_size 512 \
    --num_training_steps 10000 \
    --warmup_steps 1000 \
    --weight_decay 0 \
    --dtype bfloat16 \
    --eval_every 1000 \
    --optimizer galore_adamw

# **Benchmark 2：在 GLUE 任務上微調 RoBERTa**

run_glue.py是使用 GaLore 在 GLUE 任務上微調 RoBERTa 模型的主要腳本。範例腳本如下圖所示：

In [ ]:
!python run_glue.py \
    --model_name_or_path roberta-base \
    --task_name mrpc \
    --enable_galore \
    --lora_all_modules \
    --max_length 512 \
    --seed=1234 \
    --lora_r 4 \
    --galore_scale 4 \
    --per_device_train_batch_size 16 \
    --update_proj_gap 500 \
    --learning_rate 3e-5 \
    --num_train_epochs 30 \
    --output_dir results/ft/roberta_base/mrpc